In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [2]:
print_to_excel = True
excel_tofolder_on_Z = True

In [3]:
excel_data = pd.read_excel('report_invest_31.08.2024.xlsx', sheet_name='snapshot', header=10)
today = '2024-08-31'

In [4]:
excel_data_work = excel_data[:].reset_index(drop=True)
excel_data_work = excel_data_work.rename(columns={'ПВК|Тип прогноз. данных':'currency', 'Unnamed: 10':'Cpty', 'Балансовая единица':'Comp_code'})
# merge
excel_data_work['Cpty_code'] = merge_Mapping(col='Cpty', df=excel_data_work)
excel_data_work['Cpty_segment'] = merge_SalesUnits(excel_data_work, col='Cpty_code', merge_col='ocpSegment')
excel_data_work['Comp_segment'] = merge_SalesUnits(excel_data_work, col='Comp_code', merge_col='ocpSegment')
excel_data_work['holding'] = merge_SalesUnits(excel_data_work,'Comp_code',merge_col='holding') 
# data
excel_data_work['КалендГод/Месяц'] = excel_data_work['КалендГод/Месяц'].astype(str)
excel_data_work['КалендГод/Месяц'] = pd.to_datetime(excel_data_work['КалендГод/Месяц'], format='%m.%Y')
# to usd
excel_data_work['Заказ на поставку'] = excel_data_work['Заказ на поставку'].astype('float64').fillna(0)
# excel_data_work = add_in_currency_column(excel_data_work,'USD','currency',False,'Заказ на поставку',today)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK RU' 'SUEK RU' 'SUEK RU' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connect

In [5]:
# excel_data_work[excel_data_work.Контрагент==1000079678]

In [6]:
# create version columns
pivot_index_list=['holding','Comp_segment','Comp_code','Завод',
       'ПЕ','СПП-элемент','Контрагент','Cpty_segment','Вид проекта','Профиль проекта',
       'Проект','Вид затрат','КалендГод/Месяц', 'Календарный год',
       'Календарный месяц','currency']
excel_data_work['Версия'] = excel_data_work['Версия'].apply(lambda x: 'Версия_0' + str(x))
excel_data_version_pivot = excel_data_work.pivot_table(index=pivot_index_list, columns='Версия', values='Заказ на поставку', aggfunc=sum).reset_index().sort_values(['Comp_code','Контрагент'])
new_cols = ['Версия_045', 'Версия_075', 'Версия_035', 'Версия_085']
# excel_data_version_pivot

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15324\3410221329.py:7: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  excel_data_version_pivot = excel_data_work.pivot_table(index=pivot_index_list, columns='Версия', values='Заказ на поставку', aggfunc=sum).reset_index().sort_values(['Comp_code','Контрагент'])


In [7]:
excel_data_work['holding'].unique()

array(['EUROCHEM', 'SUEK'], dtype=object)

In [8]:
# compaire values
excel_data_version_pivot[new_cols] = excel_data_version_pivot[new_cols].fillna(0)
excel_data_version_pivot['Amount'] = np.nan
exit_data = pd.DataFrame()
for holding in ['EUROCHEM', 'SUEK']:
    holding_data = excel_data_version_pivot[excel_data_version_pivot.holding == holding]
    if holding == 'EUROCHEM':
    # Для БЕ из holding = Eurochem:
        # Если сумма платежей (значение поля " версия " = 075) меньше или равна сумме освоения (значение поля " версия " = 085), то берем платежи (значение поля " версия " = 075).
        holding_data.loc[holding_data['Версия_075']<=holding_data['Версия_085'], 'Amount'] = holding_data.loc[holding_data['Версия_075']<=holding_data['Версия_085'], 'Версия_075']
        # Если сумма платежей (значение поля " версия " = 075) больше суммы освоения (значение поля " версия " = 085), то берем сумму освоения (значение поля " версия " = 085).
        holding_data.loc[holding_data['Версия_075']>holding_data['Версия_085'], 'Amount'] = holding_data.loc[holding_data['Версия_075']>holding_data['Версия_085'], 'Версия_085']
    if holding == 'SUEK':
    # Для БЕ из holding = SUEK:
        # Если сумма платежей (значение поля " версия " = 045) меньше или равна сумме освоения (значение поля " версия " = 035), то берем платежи (значение поля " версия " = 045).
        holding_data.loc[holding_data['Версия_045']<=holding_data['Версия_035'], 'Amount'] = holding_data.loc[holding_data['Версия_045']<=holding_data['Версия_035'], 'Версия_045']
        # Если сумма платежей (значение поля " версия " = 045) больше суммы освоения (значение поля " версия " = 035), то берем сумму освоения (значение поля " версия " = 035).
        holding_data.loc[holding_data['Версия_045']>holding_data['Версия_035'], 'Amount'] = holding_data.loc[holding_data['Версия_045']>holding_data['Версия_035'], 'Версия_035']
    exit_data = pd.concat([exit_data, holding_data])

In [9]:
exit_data = add_in_currency_column(exit_data, CCY_to='USD', col_with_CCY='currency', col_with_VAL='Amount', DATE=today, date_is_column=False)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:158: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f'Coef_to_{CCY_to}'] = df.date_CCY_from.replace(coef_dict).fillna(0)


In [15]:
# Аня, в файлик по инвестициям надо добавить еще одну страницу, куда положить агрегированный набор данных по сокращенному списку полей. 
# Поля следующие. По расчётным полям amount поставить фильтр <>0
pivot_data = exit_data[exit_data.Amount!=0]
pivot_data = pivot_data.groupby(as_index=False, by=['holding','Comp_code','Comp_segment','currency','Календарный год','Календарный месяц','Cpty_segment','КалендГод/Месяц'])[['Amount','Amount_in_USD']].agg(sum)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15324\2994966810.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_data = pivot_data.groupby(as_index=False, by=['holding','Comp_code','Comp_segment','currency','Календарный год','Календарный месяц','Cpty_segment','КалендГод/Месяц'])[['Amount','Amount_in_USD']].agg(sum)


In [16]:
### to excel
for holding in ['EUROCHEM', 'SUEK']:
    data = 0
    if print_to_excel == True:
        Output_file = f'{str(date.today())}_Invest_nv_{holding}.xlsx'
        if excel_tofolder_on_Z == True:
            Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
            Output_file = Output_path + Output_file
        data = exit_data[exit_data.holding==holding]
        data.to_excel(Output_file, sheet_name='Invest', index=False)
        new_list(pivot_data[pivot_data.holding==holding], Output_file, sheet_name='pivot')